In [1]:
import sys
sys.path.append('..')
from utils import *
sys.path.append('../attacks/')
from fgsm import *
sys.path.append('../models/')
from torch import nn
import torch
from torchvision import transforms
import numpy as np
from torchvision import datasets
from torch import norm
from tqdm import tqdm
import cv2
import pandas as pd
import numpy as np

In [2]:
imgs = datasets.CIFAR10(root='/home/hancy/dataset/',train=False,transform=transforms.ToTensor(),download=True)

Files already downloaded and verified


In [3]:
model = torch.load('../models/pts/cifar_alexnet.pt')
model = model.eval()
model = model.cuda()

In [4]:
wm = cv2.imread('../img/logo.jpg')
wm = cv2.cvtColor(wm,cv2.COLOR_BGR2RGB)
wm = cv2.resize(wm,dsize=(32,32),fx=1,fy=1,interpolation=cv2.INTER_LINEAR)
wm = transforms.ToTensor()(wm).cuda()

In [5]:
def attack_and_record(filename: str,model: nn.Module, imgs:datasets, wm: Tensor):
    block_size = 4
    alpha = 0.1
    beta = 5/255
    l1 = 0.001
    l2 = 0.001
    s_a = 0.000005
    s_b = 0.0001
    N = 10
    beta_max = 8/255
    with open(filename,'a') as file:
        for i in tqdm(range(len(imgs)), desc='Processing'):
            img = imgs[i][0].cuda()
            target = torch.tensor([imgs[i][1]]).cuda()
            pred_label = model(img.unsqueeze(0)).argmax().item()
            perd_img,wm_extracted,a_res,b_res = fgsm_wm_opti(img,target,wm,model,alpha,beta,block_size,N,l1,l2,s_a,s_b,beta_max)
            res = model(perd_img.unsqueeze(0))
            perd_label = res.argmax().item()
            wm_l2_norm = norm(wm_extracted-wm).item()
            img_l2_norm = norm(perd_img-img).item()
            file.write('{},{},{},{},{},{},{},{}\n'.format(i,imgs[i][1],pred_label,perd_label,wm_l2_norm,img_l2_norm,a_res,b_res))

In [6]:
filename = '/home/hancy/code/adv_wm/atk_result/fgm_wm_cifar_alexnet2210041547.txt'
attack_and_record(filename,model,imgs,wm)

Processing:   0%|          | 0/10000 [00:02<?, ?it/s]


ValueError: not enough values to unpack (expected 4, got 2)

In [33]:
def attack_and_record_direct(filename: str,model: nn.Module, imgs:datasets, wm: Tensor):
    block_size = 4
    alpha = 0.1
    beta = 5/255
    with open(filename,'a') as file:
        for i in tqdm(range(len(imgs)), desc='Processing'):
            img = imgs[i][0].cuda()
            target = torch.tensor([imgs[i][1]]).cuda()
            pred_label = model(img.unsqueeze(0)).argmax().item()
            perd_img = fgsm_direct(img,target,wm,model,alpha,beta,block_size)
            wm_extracted = extract_wm(img,perd_img,alpha,block_size)
            res = model(perd_img.unsqueeze(0))
            perd_label = res.argmax().item()
            wm_l2_norm = norm(wm_extracted-wm).item()
            img_l2_norm = norm(perd_img-img).item()
            file.write('{},{},{},{},{},{}\n'.format(i,imgs[i][1],pred_label,perd_label,wm_l2_norm,img_l2_norm))

In [34]:
filename = '/home/hancy/code/adv_wm/atk_result/fgsm_direct_cifar_alexnet2210011644.txt'
attack_and_record_direct(filename,model,imgs,wm)

Processing: 100%|██████████| 10000/10000 [01:56<00:00, 85.60it/s]


In [31]:
check_result(filename,True)

2489
6.7830084703008735
1.423986613202095


In [16]:
direct_cols = ['index','label','pred_label','perd_label','wm_l2','img_l2']

In [17]:
res = pd.read_csv(filename,names=direct_cols,header=None)
print((res['label'] == res['perd_label']).sum())
print(res['wm_l2'].sum() / res['wm_l2'].count())
print(res['img_l2'].sum() / res['img_l2'].count())

5027
4.583950789248188
0.6742284369796514


In [35]:
def check_result(filename,direct_mode=False):
    cols = ['index','label','pred_label','perd_label','wm_l2','img_l2','alpha','beta']
    if(direct_mode):
        cols = ['index','label','pred_label','perd_label','wm_l2','img_l2']
    res = pd.read_csv(filename,names=cols,header=None)
    print((res['label'] == res['pred_label']).sum())
    print((res['label'] == res['perd_label']).sum())
    print(res['wm_l2'].sum() / res['wm_l2'].count())
    print(res['img_l2'].sum() / res['img_l2'].count())

In [36]:
check_result(filename,True)

7204
1491
11.004513784521551
1.665715494120121


In [44]:
check_result(filename,False)

7204
1630
10.425928785258119
1.502983011484146
